## Clasificación de botellas de vidrio y de plastico

In [1]:
import sys
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K
import pandas as pd
import numpy as np

## Importamos los datos

In [10]:
K.clear_session()
data_entrenamiento = './data/entrenamiento' #Cargamos data de entrenamiento
data_validacion = './data/validacion' # #Cargamos data para testear
todos_datos='./data/todo'

 
# Parametros de la red
 
epocas=10 
longitud, altura = 150, 150
batch_size = 32
pasos = 500
validation_steps = 300
filtrosConv1 = 32 # filtros que aplicamos en cada convolucion  desp de 1ra cambia la prof a 64
filtrosConv2 = 64 # filtros que aplicamos en cada convolucion 
tamano_filtro1 = (3, 3) 
tamano_filtro2 = (2, 2)

#El Max-Pooling es un proceso de discretización basado en muestras. (su papel es reducir el tamaño de las imágenes)
tamano_pool = (2, 2)
clases = 2 #cantidad de clases a entrenar
lr = 0.0004 #learning rate


## Transformamos las imagenes

In [11]:
#Preparamos nuestras imagenes
entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255, # los calores quedaran entre 0 y 1
    shear_range=0.2, #  inclina las imagenes para variar
    zoom_range=0.2, # algunas les hace zoom y otras no 
    horizontal_flip=True)# Toma imagen y la invierte para que la red aprenda direccionalidad)

test_datagen = ImageDataGenerator(rescale=1. / 255) # datos de validacion solo rescalamos.



entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size, # tamaño del batch
    class_mode='categorical') # categoria en vidrio y plastico



validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')


all_data= test_datagen.flow_from_directory(
    todos_datos,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')



Found 8132 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 10132 images belonging to 2 classes.


creamos generador Nuestras imagenes son grandes (por ejemplo 500x500). En estos casos, intentar entrenar el modelo requiere de mucha memoria que no tenemos, por lo que es necesario dividir el proceso de entrenamiento en bloques (o batchs, en inglés)(en nuestro caso lo definimos en 32) de menor tamaño de imágenes.Para ello, Keras cuenta con la clase ImageDataGenerator, que nos permite generar dichos bloques, además de realizar la técnica llamada data augmentation.

## Creamos la Red

In [4]:
 Resumen=[]
funAct=["sigmoid","relu","selu","softmax","softplus","elu","tanh"]
for i in funAct:
    cnn = Sequential() # definimos
    cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding ="same",\
                          input_shape=(longitud, altura, 3), activation=i))
    print("FUNCION DE ACTIVACION: =======> ",[i])

    #Agregamos capa de pooling  (su papel es reducir el tamaño de las imágenes)
    cnn.add(MaxPooling2D(pool_size=tamano_pool))

    cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same"))
    cnn.add(MaxPooling2D(pool_size=tamano_pool))

    cnn.add(Flatten()) # aplicamos faltten
    cnn.add(Dense(256, activation=i))

    cnn.add(Dropout(0.5)) #en cada iteracion apagamos de manera aleatorio el 50% de las neuronas
    #sirbe para darle "plasticidad" para que aprenda caminos diferentes y no siempre el mismo en cada iteracion

    cnn.add(Dense(clases, activation='softmax'))

    cnn.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
    
    
    cnn.fit(
        entrenamiento_generador,
        steps_per_epoch=pasos,
        epochs=epocas,
        validation_data=validacion_generador,
        validation_steps=validation_steps)
    df_history= pd.DataFrame(cnn.history.history)
    array_history=np.array(df_history)
    print("BEST AC "+  str("{:.4f}".format(df_history.val_accuracy.max())) + " --- VAL LOSS: "+str("{:.4f}".format(df_history.val_loss.min())))
    print("EPOCH: "+ str(np.argmax(array_history[:,3])))
    Resumen.append(i + " - BEST AC "+  str("{:.4f}".format(df_history.val_accuracy.max())) + " --- VAL LOSS: "+str("{:.4f}".format(df_history.val_loss.min()))+ " - EPOCH: "+ str(np.argmax(array_history[:,3])))
    print("___________________________________________________________________________________")


FUNCION DE ACTIVACION: =======>  ['sigmoid']
Epoch 1/10
500/500 [==============================] - 217s 434ms/step - loss: 1.0019 - accuracy: 0.4982 - val_loss: 0.6933 - val_accuracy: 0.4980
Epoch 2/10
500/500 [==============================] - 216s 432ms/step - loss: 0.7044 - accuracy: 0.5012 - val_loss: 0.7025 - val_accuracy: 0.4975
Epoch 3/10
500/500 [==============================] - 218s 437ms/step - loss: 0.7001 - accuracy: 0.5009 - val_loss: 0.6933 - val_accuracy: 0.4993
Epoch 4/10
500/500 [==============================] - 216s 433ms/step - loss: 0.7004 - accuracy: 0.5028 - val_loss: 0.6949 - val_accuracy: 0.5003
Epoch 5/10
500/500 [==============================] - 216s 432ms/step - loss: 0.7002 - accuracy: 0.5048 - val_loss: 0.7055 - val_accuracy: 0.4998
Epoch 6/10
500/500 [==============================] - 219s 439ms/step - loss: 0.7011 - accuracy: 0.5041 - val_loss: 0.6933 - val_accuracy: 0.4996
Epoch 7/10
500/500 [==============================] - 223s 446ms/step - loss: 0

In [8]:
 Resumen 

['sigmoid - BEST AC 0.5023 --- VAL LOSS: 0.6933 - EPOCH: 7',
 'relu - BEST AC 0.9777 --- VAL LOSS: 0.0650 - EPOCH: 7',
 'selu - BEST AC 0.9287 --- VAL LOSS: 0.1845 - EPOCH: 9',
 'softmax - BEST AC 0.7708 --- VAL LOSS: 0.5477 - EPOCH: 9',
 'softplus - BEST AC 0.5025 --- VAL LOSS: 0.6931 - EPOCH: 3',
 'elu - BEST AC 0.9354 --- VAL LOSS: 0.1562 - EPOCH: 9',
 'tanh - BEST AC 0.7380 --- VAL LOSS: 0.5143 - EPOCH: 7']

['sigmoid - BEST AC 0.5023 --- VAL LOSS: 0.6933 - EPOCH: 7',                       
 'relu - BEST AC 0.9777 --- VAL LOSS: 0.0650 - EPOCH: 7',                                 
 'selu - BEST AC 0.9287 --- VAL LOSS: 0.1845 - EPOCH: 9',                            
 'softmax - BEST AC 0.7708 --- VAL LOSS: 0.5477 - EPOCH: 9',                               
 'softplus - BEST AC 0.5025 --- VAL LOSS: 0.6931 - EPOCH: 3',                              
 'elu - BEST AC 0.9354 --- VAL LOSS: 0.1562 - EPOCH: 9',                               
 'tanh - BEST AC 0.7380 --- VAL LOSS: 0.5143 - EPOCH: 7']                                   

Claramente los mejores resultados lo obtuvimos con relu, ademas de eso podemos ver que solo le tomo 7 epocas asique reducimos el tiempo de entrenamiento

Entrenamos los modelos con relu

In [19]:
cnn2 = Sequential() # definimos
cnn2.add(Convolution2D(filtrosConv1, tamano_filtro1, padding ="same",\
                      input_shape=(longitud, altura, 3), activation='relu'))

#Agregamos capa de pooling 
cnn2.add(MaxPooling2D(pool_size=tamano_pool))

cnn2.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same"))
cnn2.add(MaxPooling2D(pool_size=tamano_pool))

cnn2.add(Flatten()) # aplicamos faltten
cnn2.add(Dense(256, activation='relu'))

cnn2.add(Dropout(0.5)) #en cada iteracion apagamos de manera aleatorio el 50% de las neuronas
#sirbe para darle "plasticidad" para que aprenda caminos diferentes y no siempre el mismo en cada iteracion

cnn2.add(Dense(clases, activation='softmax'))

cnn2.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])


cnn2.fit_generator(
    all_data,
    steps_per_epoch=pasos,
    epochs=7)

 
 

C:\Users\lobo\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1961: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/7
500/500 [==============================] - 153s 305ms/step - loss: 0.4525 - accuracy: 0.8756
Epoch 2/7
500/500 [==============================] - 150s 301ms/step - loss: 0.1463 - accuracy: 0.9469
Epoch 3/7
500/500 [==============================] - 151s 302ms/step - loss: 0.0889 - accuracy: 0.9691
Epoch 4/7
500/500 [==============================] - 151s 302ms/step - loss: 0.0562 - accuracy: 0.9800
Epoch 5/7
500/500 [==============================] - 152s 303ms/step - loss: 0.0422 - accuracy: 0.9860
Epoch 6/7
500/500 [==============================] - 152s 305ms/step - loss: 0.0488 - accuracy: 0.9853
Epoch 7/7
500/500 [==============================] - 150s 300ms/step - loss: 0.0320 - accuracy: 0.9882


## Creamos carpeta para guardar el modelo y lo guardamos

In [5]:
target_dir = './modelo/'
if not os.path.exists(target_dir):
    os.mkdir(target_dir)
 

In [20]:
#cnn2.save('./modelo/modelo.h5') # estructura
#cnn2.save_weights('./modelo/pesos.h5') # pesos